<a href="https://colab.research.google.com/github/pink-haribo/data-science/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression

In [ ]:
ser = pd.Series([1, 2, 3], index=['a', 'b', 'c'])

ser.sum()
ser.mean()
ser.std()
ser.min()
ser.max()
ser.skew() # 왜도
ser.kurt() # 첨도
ser.std(ddof=1) # delta degree of freedom
ser.unique()
ser.idxmax() # index of max-value
ser.isin([1, 2]) # filtering value is 1 or 2

,0
a,True
b,True
c,False


In [ ]:
df = pd.DataFrame({"aa":[1, 2, 2, 3], "bb":[ 4, 4, 5, 6]})
df

,aa,bb
0,1,4
1,2,4
2,2,5
3,3,6


In [ ]:
df = pd.read_csv("./drive/MyDrive/Data-Science/data/bike.csv")
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [ ]:
df.iloc[:, :2]
df.loc[:, ["season", "holiday"]]
df.loc[:, "season":"casual"]
df["season"].value_counts()
pd.crosstab(df["season"], df["holiday"], normalize=True)
pd.crosstab(df["season"], df["holiday"], normalize=1) # col 기준 normalize
pd.crosstab(df["season"], df["holiday"], values=df["humidity"], aggfunc="mean")
df.groupby(["season", "holiday"])["humidity"].mean()
# -> pd.crosstab(df["season"], df["holiday"], values=df["humidity"], aggfunc="mean")과 동일

df["season"].unique()
df["season"].nunique()

df.sort_values("season", ascending=False)

df.loc[:, "season":"weather"].apply(func=sum, axis=1)
df.loc[:, "season":"weather"].apply(func=lambda x: round(x.mean()), axis=1)

df.isna().sum(axis=1)
df.notna()
df.fillna(value={"season":-1, "weather":df["weather"].mean()})
df.dropna(how="all")

df["humidity"].quantile(q=0.25) # 1/2 IQR
df["humidity"].quantile() == df["humidity"].median()

cond1 = df["humidity"] > 10
cond2 = df["season"] >= 2
df.loc[cond1 | cond2].shape

df["is_winter"] =  (df["season"] == 4) + 0
df.rename(columns={"weather":"weather_value"})

pd.to_datetime(df["datetime"]).dt.year

pd.get_dummies(data= df, columns=["season"], drop_first=True, dtype=int) # one-hot encoding

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,is_winter,season_2,season_3,season_4
0,2011-01-01 00:00:00,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,0,0,0
1,2011-01-01 01:00:00,0,0,1,9.02,13.635,80,0.0000,8,32,40,0,0,0,0
2,2011-01-01 02:00:00,0,0,1,9.02,13.635,80,0.0000,5,27,32,0,0,0,0
3,2011-01-01 03:00:00,0,0,1,9.84,14.395,75,0.0000,3,10,13,0,0,0,0
4,2011-01-01 04:00:00,0,0,1,9.84,14.395,75,0.0000,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,0,1,1,15.58,19.695,50,26.0027,7,329,336,1,0,0,1
10882,2012-12-19 20:00:00,0,1,1,14.76,17.425,57,15.0013,10,231,241,1,0,0,1
10883,2012-12-19 21:00:00,0,1,1,13.94,15.910,61,15.0013,4,164,168,1,0,0,1
10884,2012-12-19 22:00:00,0,1,1,13.94,17.425,61,6.0032,12,117,129,1,0,0,1


In [ ]:
df.groupby("season").sample(n=2) # 층화추출
df_train, df_test = train_test_split(df, train_size = 0.7, random_state= 123)

In [ ]:
df.reset_index()
df.set_index("datetime")

head = df.head()
tail = df.head()
pd.concat([head, tail.reset_index(drop=True)], axis=1)

pd.merge(left=head, right=tail, left_on="season", right_on="season", how="inner").head()

,datetime_x,season,holiday_x,workingday_x,weather_x,temp_x,atemp_x,humidity_x,windspeed_x,casual_x,...,workingday_y,weather_y,temp_y,atemp_y,humidity_y,windspeed_y,casual_y,registered_y,count_y,is_winter_y
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,0,1,9.84,14.395,81,0.0,3,13,16,0
1,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,0,1,9.02,13.635,80,0.0,8,32,40,0
2,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,0,1,9.02,13.635,80,0.0,5,27,32,0
3,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,0,1,9.84,14.395,75,0.0,3,10,13,0
4,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,0,1,9.84,14.395,75,0.0,0,1,1,0


In [ ]:
elec = pd.read_csv('/content/drive/MyDrive/Data-Science/data/elec_load_2017_7d.csv')
print(elec.head())

elec_melt = elec.melt(id_vars = ["YEAR", "MONTH", "DAY"])
print(elec_melt.head())

elec_melt = elec.melt(id_vars = ["YEAR", "MONTH", "DAY"], var_name="HOUR", value_name="LOAD")
print(elec_melt.head())

elec_pivot = elec_melt.pivot(index=["YEAR", "MONTH", "DAY"],
                             columns="HOUR", values="LOAD").reset_index()
print(elec_pivot.head())

   YEAR  MONTH  DAY   X1HR   X2HR   X3HR   X4HR   X5HR   X6HR   X7HR  ...  \
0  2017      1    1  60178  57862  56165  55135  54450  54401  54147  ...   
1  2017      1    2  57067  55148  53983  53571  54027  55388  57753  ...   
2  2017      1    3  66263  64233  63194  62461  62649  63396  65098  ...   
3  2017      1    4  68403  66143  65051  64245  64152  64678  66494  ...   
4  2017      1    5  68053  65801  64439  63437  63135  63250  63658  ...   

   X15HR  X16HR  X17HR  X18HR  X19HR  X20HR  X21HR  X22HR  X23HR  X24HR  
0  49213  49357  50362  53115  55205  55437  55348  55790  58195  59330  
1  72228  71945  72641  73387  72294  69989  67893  66504  67435  69275  
2  75576  75168  75791  75922  75469  73118  70769  68726  69593  71346  
3  74977  74275  74909  75344  74504  71957  69715  68155  68948  70834  
4  64767  63613  63095  63230  64253  63353  62455  62314  64350  66687  

[5 rows x 27 columns]
   YEAR  MONTH  DAY variable  value
0  2017      1    1     X1HR  6017

In [ ]:
df.corr(numeric_only=True)
df[["casual", "registered", "count"]].corr()
df[["casual", "registered", "count"]].corr(method="spearman")
df[["casual", "registered", "count"]].corr(method="pearson")

pearsonr(df["casual"], df["registered"]) # p-value < 0.05 i.e. 귀무가설/H0 기각 & 대립가설/H1 채택

df["is_sunny"] = (df["weather"] == 1) + 0
df.groupby("is_sunny")[["temp", "casual"]].corr()

temp    casual
is_sunny                           
0        temp    1.000000  0.446361
         casual  0.446361  1.000000
1        temp    1.000000  0.471053
         casual  0.471053  1.000000

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler

minmax = MinMaxScaler().fit(df[["temp", "casual"]])
minmax.data_max_
minmax.data_min_

df_minmax = minmax.transform(df[["temp", "casual"]])
df_minmax = pd.DataFrame(df_minmax, columns=["temp", "casual"])
df_minmax.head()


,temp,casual
0,0.224490,0.008174
1,0.204082,0.021798
2,0.204082,0.013624
3,0.224490,0.008174
4,0.224490,0.000000


In [ ]:
model = KMeans(n_clusters=4, random_state=123).fit(df.iloc[:, 1:])
df["cluster"] = model.labels_
print(df["cluster"].value_counts())
df.head()

cluster
0    4902
2    3507
1    1859
3     618
Name: count, dtype: int64


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,is_winter,is_sunny,cluster
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,0,1,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,0,1,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,0,1,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,0,1,0


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv('/content/drive/MyDrive/Data-Science/data/iris.csv')
df.columns = ["SL", "SW", "PL", "PW", "species"]

model = ols(formula ="PL ~ PW", data = df).fit()
df["pred"] = model.predict(df)
model.summary() # prob(f) = 4.68e-86 < 0.05 : H0 기각

model = LinearRegression().fit(X=df[["PL"]], y=df["PW"])
model.coef_, model.intercept_
df["pred"] = model.predict(df[["PL"]])

mean_absolute_error(y_true=df["PW"], y_pred=df["pred"])
mean_squared_error(y_true=df["PW"], y_pred=df["pred"]) ** 0.5 # RMSE

0.20510316719883478

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

data = pd.DataFrame({
    'y': [1, 2, 3, 4, 5],
    'X1': [10, 20, 30, 40, 50],
    'X2': [5, 4, 3, 2, 1]
})

# dmatrices를 사용해 회귀 모델용 변수 추출
y, X = dmatrices('y ~ X1 + X2', data, return_type="dataframe")
df_vif= pd.DataFrame()
df_vif["colname"] = X.columns
df_vif["VIF"] = [vif(X.values, i) for i in range(X.shape[1])]
df_vif


/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,colname,VIF
0,Intercept,0.0
1,X1,inf
2,X2,inf


In [ ]:
import numpy as np
import pandas as pd

from statsmodels.api import Logit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

df = pd.read_csv('/content/drive/MyDrive/Data-Science/data/iris.csv')
df["is_setosa"] = (df['Species'] == "setosa") + 0

model = Logit(endog=df["is_setosa"], exog=df.iloc[:, :2]).fit()
model.summary()
pred_conf = model.predict(df.iloc[:, :2])
pred_conf.apply(lambda x: 1 if x > 0.5 else 0)

model = LogisticRegression(random_state=123)
model.fit(X=df.iloc[:, :2], y = df["is_setosa"])
pred_conf = model.predict_proba(df.iloc[:, :2])

roc_auc_score(y_true=df["is_setosa"], y_score=pred_conf[:, 1])
accuracy_score(y_true=df["is_setosa"], y_pred=pred_conf.argmax(axis=1))

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


1.0

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB

df["is_setosa"] = (df["Species"] == "setosa") + 0
df["is_setosa"].value_counts(normalize=True) # prior

model = GaussianNB().fit(X = df.iloc[:, :4],
                         y = df["is_setosa"])
model.class_prior_, model.theta_

pred = model.predict_proba(df.iloc[:, :4])
pred.shape
pred = pred[:, 1]

pred_class = (pred > 0.5) + 0
accuracy_score(y_true = df["is_setosa"],
               y_pred = pred_class)

NameError: name 'df' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score

df = pd.read_csv('/content/drive/MyDrive/Data-Science/data/iris.csv')
df.head()

df["is_setosa"] = (df["Species"] == "setosa") + 0

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X = df.iloc[:, :4], y=df["is_setosa"])
model

pred = model.predict(df.iloc[:, :4])
accuracy_score(y_true=df["is_setosa"], y_pred=pred)

NameError: name 'pd' is not defined

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

df = pd.read_csv('/content/drive/MyDrive/Data-Science/data/iris.csv')
df.head()

df["is_versicolor"] = (df["Species"] == "versicolor") + 0
pd.crosstab(df["Species"], df["is_versicolor"])

model = DecisionTreeClassifier(random_state=123, max_depth=2)
model.fit(X = df.iloc[:, :4],
          y = df["is_versicolor"])
model

pred = model.predict(df.iloc[:, :4])


accuracy_score(y_true = df["is_versicolor"],
               y_pred = pred)


model = DecisionTreeRegressor(random_state= 123)
model.fit(X=df.iloc[:, :3], y=df["Petal.Width"])
pred = model.predict(df.iloc[:, :3])
pred[:4]

mean_squared_error(y_true=df["Petal.Width"], y_pred=pred)